In [1]:
import numpy as np

import lal
from lal import ComputeDetAMResponse
from lal import CachedDetectors

from lal import LALDetectorIndexLHODIFF,LALDetectorIndexLLODIFF,LALDetectorIndexVIRGODIFF

DETECTOR_SITES = {
    'H1': LALDetectorIndexLHODIFF,
    'L1': LALDetectorIndexLLODIFF,
    'V1': LALDetectorIndexVIRGODIFF
    }

import lalsimulation

import matplotlib.pyplot as plt

In [2]:
class Detector(object):
    """
    A Detector object characterises a gravitational wave (GW) interferometric detector
    """

    def __init__(self, detector):
        """
        detector  -- label string of the detector
        descriptor -- LAL descriptor
        location -- geographic location of the detector
        response -- response matrix

        """
        self.name = detector
        self.descriptor =  CachedDetectors[DETECTOR_SITES[detector]]
        self.location = lalsimulation.DetectorPrefixToLALDetector(detector).location
        self.response = lalsimulation.DetectorPrefixToLALDetector(detector).response
        
    def antenna_pattern(self, time_at_coalescence, RA, dec):
        
        gmst_rad = lal.GreenwichMeanSiderealTime(time_at_coalescence)
        """fplus = []
        fcross = []
        for (RA_val, dec_val) in zip(RA, dec):
            fplus_val,fcross_val = ComputeDetAMResponse(self.descriptor.response,
                                    RA_val, dec_val, psi, gmst_rad)
            fplus.append(fplus_val)
            fcross.append(fcross_val)

        return np.array(fplus), np.array(fcross)
        """
        fplus,fcross = ComputeDetAMResponse(self.descriptor.response,
                                    RA, dec, psi, gmst_rad)
        
        return fplus,fcross
        
        
    def time_delay_from_earth_center(self, RA, dec, time_gps):
        """ Returns the time delay from the earth center
        """
        return lal.TimeDelayFromEarthCenter(self.location,
                      float(RA), float(dec), float(time_gps))

In [39]:
#constants
msun = 4.926e-6 #s-1       #1.988*10**30 #kg
c = 1                     #3*10**8 #m/s
G = 1                     #6.674*10**(-11)#N/kg**2*m**2
parsec = 3.086*10**16/(2.998*10**8) #s

#parameters
m1 = 1.46*msun
m2 =  1.26*msun
iota = 90.*np.pi/180
v = np.cos(iota)
psi = 0.*np.pi/180
D = 40*10**6*parsec
tc = 1187008882.43
phic = 0.*np.pi/180
time_at_coalescence = tc
mchirp = (m1*m2)**(3./5.)/(m1+m2)**(1./5.)

ra = 168*np.pi/180
dec = 61*np.pi/180

In [36]:
#some useful functions

def antenna(Fplus,Fcross,v):

    output = (1+v**2)/2*Fplus-1j*v*Fcross
    
    return output


def strain(f, tau):
    
    Fplus, Fcross = detector.antenna_pattern(time_at_coalescence, ra, dec)
    
    phase = 2.*np.pi*f*tc - phic - np.pi/4. + 3./4.*(8.*np.pi*mchirp*f)**(-5./3.)
    
    h = np.e**(1j*2*np.pi*f*tau)*strain0(f)*antenna(Fplus, Fcross, v)

    return h


def strain0(f):
    
    Fplus, Fcross = detector.antenna_pattern(time_at_coalescence, ra, dec)
    
    phase = 2.*np.pi*f*tc -    phic - np.pi/4. + 3./4.*(8.*np.pi*mchirp*f)**(-5./3.)

    h0 = np.sqrt(5./24.)*np.pi**(-2./3.)*c**(-3/2)*D**(-1)*(G*mchirp)**(5./6.)*f**(-7./6.)*np.e**(1j*phase)
    
    return h0


def derivatives(detector,ra,dec,epsilon):
    #numerical derivatives of Fplus and Fcross
    
    derFplusplus = (((detector.antenna_pattern(time_at_coalescence,ra+epsilon, dec))[0]))
    derFplusminus = (((detector.antenna_pattern(time_at_coalescence,ra-epsilon, dec))[0]))
    derFplus = (derFplusplus-derFplusminus)/(2*epsilon)
    derFplus_ra = float(derFplus)


    derFcrossplus = (((detector.antenna_pattern(time_at_coalescence,ra+epsilon, dec))[1]))
    derFcrossminus = (((detector.antenna_pattern(time_at_coalescence,ra-epsilon, dec))[1]))
    derFcross = (derFcrossplus-derFcrossminus)/(2*epsilon)
    derFcross_ra = float(derFcross)


    derFplusplus = (((detector.antenna_pattern(time_at_coalescence,ra, dec+epsilon))[0]))
    derFplusminus = (((detector.antenna_pattern(time_at_coalescence,ra, dec-epsilon))[0]))
    derFplus = (derFplusplus-derFplusminus)/(2*epsilon)
    derFplus_dec = float(derFplus)


    derFcrossplus = (((detector.antenna_pattern(time_at_coalescence,ra, dec+epsilon))[1]))
    derFcrossminus = (((detector.antenna_pattern(time_at_coalescence,ra, dec-epsilon))[1]))
    derFcross = (derFcrossplus-derFcrossminus)/(2*epsilon)
    derFcross_dec = float(derFcross)
    
    return derFplus_ra, derFcross_ra,derFplus_dec,derFcross_dec 

def derivativestau(detector,ra,dec,epsilon):
    
    #numerical derivative for tau
    dertauplus = detector.time_delay_from_earth_center(ra+epsilon, dec, time_at_coalescence)
    dertauminus = detector.time_delay_from_earth_center(ra-epsilon, dec, time_at_coalescence)
    dertau_ra = (dertauplus - dertauminus)/(2*epsilon)
    
    dertauplus = detector.time_delay_from_earth_center(ra, dec+epsilon, time_at_coalescence)
    dertauminus = detector.time_delay_from_earth_center(ra, dec-epsilon, time_at_coalescence)
    dertau_dec = (dertauplus - dertauminus)/(2*epsilon)
    
    dertauplus = detector.time_delay_from_earth_center(ra, dec, time_at_coalescence+epsilon*100)
    dertauminus = detector.time_delay_from_earth_center(ra, dec, time_at_coalescence-epsilon*100)
    dertau_tc = (dertauplus - dertauminus)/(2*epsilon)  
    
    return dertau_ra, dertau_dec, dertau_tc

def derivativepsi(detector,ra,dec,epsilon):
    
    #numerical of fcross and fplus with respect to psi 
    fplusplus, fcrossplus = ComputeDetAMResponse(detector.descriptor.response, ra, dec, psi+epsilon, tc)
    fplusminus, fcrossminus = ComputeDetAMResponse(detector.descriptor.response, ra, dec, psi-epsilon, tc)

    dfplus_dpsi = (fplusplus-fplusminus)/(2*epsilon)
    dfcross_dpsi = (fcrossplus-fcrossminus)/(2*epsilon)
    
    return dfplus_dpsi, dfcross_dpsi

In [9]:
detectors = ['H1', 'L1', 'V1']
#print strain0(10.)

In [42]:
#Fisher matrix for 8 unknowns: D, v=cosi, psi, theta, phi, tc, phic, mchirp
fval = np.arange(1000)/1. + 20

df = (fval.max()-fval.min())/len(fval)

n = 8

# espilon for the numerical derivatives of Fplus and Fcross
epsilon = 1e-10

F = np.zeros([n,n])

for det in detectors:

    if det is 'L1':
        print 'Livingston'
        Sval = (0.8e-22*(14./(0.08+fval))**2)**2+0.67e-23**2+((fval/2000.)*3.e-23)**2
        detector = Detector('L1')
    if det is 'H1':
        print 'Hanford'
        Sval = ((1e-22*(24./(0.1+fval))**2)**2+0.7e-23**2+((fval/2000.)*3.4e-23)**2)
        detector = Detector('H1')
    if det is 'V1':
        print 'Virgo'
        Sval = (1e-22*(35./(0.1+fval))**2)**2+0.9e-23**2+((fval/2000.)*15e-23)**2
        detector = Detector('V1')
    
    tau = time_delay = detector.time_delay_from_earth_center(ra, dec, time_at_coalescence)
    
    Fplus, Fcross = detector.antenna_pattern(time_at_coalescence, ra, dec)

    #numerical derivatives
    derFplus_ra, derFcross_ra, derFplus_dec, derFcross_dec  = derivatives(detector,ra,dec,epsilon)
    dertau_ra, dertau_dec, dertau_tc = derivativestau(detector,ra,dec,epsilon)
    derFplus_psi, derFcross_psi = derivativepsi(detector,ra,dec,epsilon)


    for f,S in zip(fval,Sval):
        for i in range(n):
            if(i==0):
                dfdpi = -strain(f,tau)/D
            if(i==1):
                dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(v*Fplus-1j*Fcross)
            if(i==2):
                dfdpi = strain(f,tau)*(5/(6*mchirp)-1j*5/4*(8*np.pi*f)**(-5/3)*mchirp**(-8/3))
            if(i==3):
                dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_ra*(1+v**2)/2-1j*v*derFcross_ra)+strain(f,tau)*2*np.pi*1j*f*dertau_ra
            if(i==4):
                dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_dec*(1+v**2)/2-1j*v*derFcross_dec)+strain(f,tau)*2*np.pi*1j*f*dertau_dec
            if(i==5):
                dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*((1+v**2)/2*derFplus_psi-1j*v*derFcross_psi)
            if(i==6):
                dfdpi = 2j*np.pi*f*(1+dertau_tc)*strain(f,tau)
            if(i==7):
                dfdpi = -1j*strain(f,tau)
            
            for j in range(n):
                if(j==0):
                    dfdpj = -strain(f,tau)/D
                if(j==1):
                    dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(v*Fplus-1j*Fcross)
                if(j==2):
                    dfdpj = strain(f,tau)*(5/(6*mchirp)-1j*5/4*(8*np.pi*f)**(-5/3)*mchirp**(-8/3))
                if(j==3):
                    dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_ra*(1+v**2)/2-1j*v*derFcross_ra)+strain(f,tau)*2*np.pi*1j*f*dertau_ra
                if(j==4):
                    dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_dec*(1+v**2)/2-1j*v*derFcross_dec)+strain(f,tau)*2*np.pi*1j*f*dertau_dec
                if(j==5):
                    dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*((1+v**2)/2*derFplus_psi-1j*v*derFcross_psi)
                if(j==6):
                    dfdpj = 2j*np.pi*f*(1+dertau_tc)*strain(f,tau)
                if(j==7):
                    dfdpj = -1j*strain(f,tau)

                F[i,j] += 4*S**(-1)*(dfdpi.real*dfdpj.real+dfdpi.imag*dfdpj.imag)*df
                
                #F[i,j] += 2*S**(-1)*(dfdpi*dfdpj.conjugate()+dfdpi.conjugate()*dfdpj)*df

I = (np.mat(F).I)
print I

print 'The relative distance error is {0}'.format(np.sqrt(I[0,0]/D**2))
print 'delta cos(iota) = {0}'.format(np.sqrt(I[1,1]))
print 'The relative mchirp error is {0}'.format(np.sqrt(I[2,2]/mchirp**2))
print 'delta ra error = {0}'.format(np.sqrt(I[3,3]))
print 'delta dec error = {0}'.format(np.sqrt(I[4,4]))
print 'delta psi error = {0}'.format(np.sqrt(I[5,5]))
print "delta t_c = {0}".format(np.sqrt(I[6,6]))
print 'delta phi_c error = {0}'.format(np.sqrt(I[7,7]))

Hanford
Livingston
Virgo
[[  2.10923340e+31   9.09056443e+14   4.73072947e+04  -1.15398404e+15
   -1.04363584e+14  -2.08606405e+15   4.28449343e+12   1.65184031e+14]
 [  9.09056443e+14   6.73352299e-02   2.93766655e-12  -6.49001560e-02
   -1.62851487e-03  -8.97802353e-02   3.04636242e-04   3.43764741e-02]
 [  4.73072947e+04   2.93766655e-12   2.84906089e-21  -3.30353242e-12
   -1.56076610e-13  -4.71931608e-12  -3.77295697e-14  -3.16880952e-11]
 [ -1.15398404e+15  -6.49001560e-02  -3.30353242e-12   7.87062341e-02
    5.62625461e-03   1.16330256e-01  -3.07173508e-04  -1.49170939e-02]
 [ -1.04363584e+14  -1.62851487e-03  -1.56076610e-13   5.62625461e-03
    1.95150917e-03   1.14774894e-02  -6.42883058e-06   2.72409536e-03]
 [ -2.08606405e+15  -8.97802353e-02  -4.71931609e-12   1.16330256e-01
    1.14774894e-02   2.10005479e-01  -4.22321355e-04  -1.43127635e-02]
 [  4.28449343e+12   3.04636242e-04  -3.77295697e-14  -3.07173508e-04
   -6.42883058e-06  -4.22321355e-04   3.03341848e-06   1.04

In [26]:
#tests on ordres de grandeur
print (strain(40,tau))
print (strain0(40))

for i in range(8):
    print F[i,i]

(8.73924234717e-24+2.36360842583e-23j)
(-2.26205634697e-23-2.01406436658e-23j)
1.77958554603e-17
339752865.135
9.49887393655e+27
946652623.868
12668863869.6
1814431343.99
6.48009216254e+13
301694531.88


In [ ]:
#Fisher matrix for 8 unknowns: D, v=cosi, psi, theta, phi, tc, phic, mchirp
fval = np.arange(2020)/1. + 20
 
df = (fval.max()-fval.min())/len(fval)

n = 8

# espilon for the numerical derivatives of Fplus and Fcross
epsilon = 1e-10

n_ra = 50

ra_plot = np.linspace(-np.pi, np.pi, n_ra)
dec_plot= np.linspace(-np.pi/2, np.pi/2, n_ra)

ramesh, decmesh = np.meshgrid(ra_plot,dec_plot)

relDerror = []

for (ra,dec) in zip(ramesh.flatten(),decmesh.flatten()):
    F = np.zeros([n,n])
    for det in detectors:

        if det is 'L1':
            #print 'Livingston'
            Sval = (0.8e-22*(14./(0.08+fval))**2)**2+0.67e-23**2+((fval/2000.)*3.e-23)**2
            detector = Detector('L1')
        if det is 'H1':
            #print 'Hanford'
            Sval = ((1e-22*(24./(0.1+fval))**2)**2+0.7e-23**2+((fval/2000.)*3.4e-23)**2)
            detector = Detector('H1')
        if det is 'V1':
            #print 'Virgo'
            Sval = (1e-22*(35./(0.1+fval))**2)**2+0.9e-23**2+((fval/2000.)*15e-23)**2
            detector = Detector('V1')

        tau = time_delay = detector.time_delay_from_earth_center(ra, dec, time_at_coalescence)

        Fplus, Fcross = detector.antenna_pattern(time_at_coalescence, ra, dec)

        #numerical derivatives
        derFplus_ra, derFcross_ra, derFplus_dec, derFcross_dec  = derivatives(detector,ra,dec,epsilon)
        dertau_ra, dertau_dec, dertau_tc = derivativestau(detector,ra,dec,epsilon)
        derFplus_psi, derFcross_psi = derivativepsi(detector,ra,dec,epsilon)


        for f,S in zip(fval,Sval):
            for i in range(n):
                if(i==0):
                    dfdpi = -strain(f,tau)/D
                if(i==1):
                    dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(v*Fplus-1j*Fcross)
                if(i==2):
                    dfdpi = strain(f,tau)*(5/(6*mchirp)-1j*5/4*(8*np.pi*f)**(-5/3)*mchirp**(-8/3))
                if(i==3):
                    dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_ra*(1+v**2)/2-1j*v*derFcross_ra)+strain(f,tau)*2*np.pi*1j*f*dertau_ra
                if(i==4):
                    dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_dec*(1+v**2)/2-1j*v*derFcross_dec)+strain(f,tau)*2*np.pi*1j*f*dertau_dec
                if(i==5):
                    dfdpi = np.e**(1j*2*np.pi*f*tau)*strain0(f)*((1+v**2)/2*derFplus_psi-1j*v*derFcross_psi)
                if(i==6):
                    dfdpi = 2j*np.pi*f*(1+dertau_tc)*strain(f,tau)
                if(i==7):
                    dfdpi = -1j*strain(f,tau)

                for j in range(n):
                    if(j==0):
                        dfdpj = -strain(f,tau)/D
                    if(j==1):
                        dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(v*Fplus-1j*Fcross)
                    if(j==2):
                        dfdpj = strain(f,tau)*(5/(6*mchirp)-1j*5/4*(8*np.pi*f)**(-5/3)*mchirp**(-8/3))
                    if(j==3):
                        dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_ra*(1+v**2)/2-1j*v*derFcross_ra)+strain(f,tau)*2*np.pi*1j*f*dertau_ra
                    if(j==4):
                        dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*(derFplus_dec*(1+v**2)/2-1j*v*derFcross_dec)+strain(f,tau)*2*np.pi*1j*f*dertau_dec
                    if(j==5):
                        dfdpj = np.e**(1j*2*np.pi*f*tau)*strain0(f)*((1+v**2)/2*derFplus_psi-1j*v*derFcross_psi)
                    if(j==6):
                        dfdpj = 2j*np.pi*f*(1+dertau_tc)*strain(f,tau)
                    if(j==7):
                        dfdpj = -1j*strain(f,tau)

                    F[i,j] += 4*S**(-1)*(dfdpi.real*dfdpj.real+dfdpi.imag*dfdpj.imag)*df

                    #F[i,j] += 2*S**(-1)*(dfdpi*dfdpj.conjugate()+dfdpi.conjugate()*dfdpj)*df

    I = (np.mat(F).I)
    
    relDerror.append(np.sqrt(I[0,0])/D)
    

relDerror = np.array(relDerror)
relDerror = relDerror.reshape(ramesh.shape)

print relDerror

np.savetxt('relDerror5050iota90.txt',relDerror)
    


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in sqrt
